In [1]:
import pandas as pd
import matplotlib as plt
import re
from IPython.display import FileLink

In [2]:
filepath = "C:/Users/bkpug/Downloads/archive/fifa21 raw data v2.csv"
df = pd.read_csv(filepath)
df.head()

C:\Users\bkpug\AppData\Local\Temp\ipykernel_13456\818199989.py:2: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595


In [3]:
club = df['Club'].str.replace('\n', '', regex=False)
club.head()

0           FC Barcelona
1               Juventus
2        Atlético Madrid
3        Manchester City
4    Paris Saint-Germain
Name: Club, dtype: object

In [4]:
# Remove Kg from the Weight column to convert it to an Int
df['Weight'] = df['Weight'].str.replace('kg', '')

# Extract the numeric value and unit for lbs
df['Weight_Value'] = df['Weight'].str.extract('(\d+)', expand=False).astype(float)
df['Unit'] = df['Weight'].str.extract('(kg|lbs)', expand=False)

# Convert unit for lbs into kg
def convert_weight(row):
    if row['Unit'] == 'lbs':
        # Convert lbs to kg (1 lb = 0.453592 kg)
        return row['Weight_Value'] * 0.453592
    return row['Weight_Value']  # In case of no conversion

# Apply conversion to a new column
df['Converted_Weight'] = df.apply(convert_weight, axis=1)

# Convert original weight column into converted new column
df['Weight'] = df['Converted_Weight']

# Convert Weight column into int and check by checking a random column
df['Weight'] = df['Weight'].astype(int)
df['Weight'].iloc[866]

88

In [5]:
# Function to convert height in feet to cm
def convert_to_cm(height):
    # If the height is in feet (like "6'2")
    match = re.match(r"(\d+)'(\d+)", height)  # Matches height in feet (e.g., "6'2")
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        # Convert feet to cm (1 foot = 30.48 cm), and inches to cm (1 inch = 2.54 cm)
        total_cm = feet * 30.48 + inches * 2.54
        return total_cm
    
    # If the height is already in cm (like "180cm")
    match = re.match(r"(\d+)cm", height)  # Matches height in cm (e.g., "180cm")
    if match:
        return int(match.group(1))  # Return the numeric value in cm
    
    return None  # In case the height format is unknown

# Apply the conversion function to the 'Height' column
df["Height_cm"] = df["Height"].apply(convert_to_cm)

# Change the full column for ['Height'] to the converted column
df['Height'] = df['Height_cm']

# Change the dtype of the column
df['Height'] = df['Height'].astype(int)
# Test
df['Height'].iloc[886]
df['Height'].head()


0    170
1    187
2    188
3    181
4    175
Name: Height, dtype: int32

In [6]:
# Function to convert 'm' or 'K' values to full integer
def convert_to_full_value(value):
    value_without_euro = str(value).replace('€', '').strip()

    if '€' in value_without_euro:
        print(f"Warning: There is still a € symbol in the value '{value_without_euro}'")
    else:
        print(f"€ symbol removed. Cleaned value: {value_without_euro}")
    # Match for values in the form of €<number>m (millions) or €<number>K (thousands)
    match_million = re.match(r'€(\d+(\.\d+)?)m', value, re.IGNORECASE)  # For 'm' or 'M'
    match_thousand = re.match(r'€(\d+(\.\d+)?)k', value, re.IGNORECASE)  # For 'k' or 'K'
    
    if match_million:
        # Extract the numeric value, convert to float, multiply by 1,000,000, and return as int
        numeric_value = match_million.group(1)
        try:
            full_value = int(float(numeric_value) * 1_000_000)
            return full_value
        except ValueError as e:
            print(f"Error converting value '{numeric_value}' to float (millions): {e}")
            return None  # Or handle error as needed (e.g., return 0)
    
    elif match_thousand:
        # Extract the numeric value, convert to float, multiply by 1,000, and return as int
        numeric_value = match_thousand.group(1)
        try:
            full_value = int(float(numeric_value) * 1_000)
            return full_value
        except ValueError as e:
            print(f"Error converting value '{numeric_value}' to float (thousands): {e}")
            return None  # Or handle error as needed (e.g., return 0)

    else:
        # If no match, check if it's just a raw number (like '0' or '100')
        try:
            return int(value_without_euro)  # Convert pure numbers like '0' or '100' directly to integer
        except ValueError:
            print(f"No match and unable to convert value: {value_without_euro}")
            return None  # Or handle error as needed (e.g., return 'Invalid')

# Display the updated DataFrame


df['Full_Value'] = df['Value'].apply(convert_to_full_value)
df['Value'] = df['Full_Value']

€ symbol removed. Cleaned value: 103.5M
€ symbol removed. Cleaned value: 63M
€ symbol removed. Cleaned value: 120M
€ symbol removed. Cleaned value: 129M
€ symbol removed. Cleaned value: 132M
€ symbol removed. Cleaned value: 111M
€ symbol removed. Cleaned value: 120.5M
€ symbol removed. Cleaned value: 102M
€ symbol removed. Cleaned value: 185.5M
€ symbol removed. Cleaned value: 110M
€ symbol removed. Cleaned value: 113M
€ symbol removed. Cleaned value: 120.5M
€ symbol removed. Cleaned value: 90.5M
€ symbol removed. Cleaned value: 82M
€ symbol removed. Cleaned value: 17.5M
€ symbol removed. Cleaned value: 83.5M
€ symbol removed. Cleaned value: 33.5M
€ symbol removed. Cleaned value: 83.5M
€ symbol removed. Cleaned value: 114.5M
€ symbol removed. Cleaned value: 78M
€ symbol removed. Cleaned value: 103M
€ symbol removed. Cleaned value: 109M
€ symbol removed. Cleaned value: 92M
€ symbol removed. Cleaned value: 109M
€ symbol removed. Cleaned value: 10M
€ symbol removed. Cleaned value: 76.5M
€

In [7]:
df[['Month', 'Day_Year']] = df['Joined'].str.split(' ', expand=True, n=1)

df = df[df['Month'].notnull() & df['Day_Year'].notnull()]

df['Day'] = df['Day_Year'].str.replace(',', '').str.split().str[0]

df['Year'] = df['Day_Year'].str.split().str[1]

df.drop(columns=['Day_Year'], inplace=True)

In [8]:
df['Wage'] = df['Wage'].apply(convert_to_full_value)
df['Wage'] = df['Wage'].astype(int)

€ symbol removed. Cleaned value: 560K
€ symbol removed. Cleaned value: 220K
€ symbol removed. Cleaned value: 125K
€ symbol removed. Cleaned value: 370K
€ symbol removed. Cleaned value: 270K
€ symbol removed. Cleaned value: 240K
€ symbol removed. Cleaned value: 250K
€ symbol removed. Cleaned value: 160K
€ symbol removed. Cleaned value: 160K
€ symbol removed. Cleaned value: 260K
€ symbol removed. Cleaned value: 210K
€ symbol removed. Cleaned value: 250K
€ symbol removed. Cleaned value: 310K
€ symbol removed. Cleaned value: 250K
€ symbol removed. Cleaned value: 130K
€ symbol removed. Cleaned value: 350K
€ symbol removed. Cleaned value: 300K
€ symbol removed. Cleaned value: 300K
€ symbol removed. Cleaned value: 270K
€ symbol removed. Cleaned value: 190K
€ symbol removed. Cleaned value: 145K
€ symbol removed. Cleaned value: 190K
€ symbol removed. Cleaned value: 195K
€ symbol removed. Cleaned value: 220K
€ symbol removed. Cleaned value: 100K
€ symbol removed. Cleaned value: 140K
€ symbol rem

In [9]:
df.drop(columns=['Weight_Value', 'Height_cm', 'Loan Date End', 'Unit', 'Converted_Weight', 'Value'], inplace=True)

In [10]:
pd.set_option('display.max_rows', None)
df.dtypes

ID                   int64
Name                object
LongName            object
photoUrl            object
playerUrl           object
Nationality         object
Age                  int64
↓OVA                 int64
POT                  int64
Club                object
Contract            object
Positions           object
Height               int32
Weight               int32
Preferred Foot      object
BOV                  int64
Best Position       object
Joined              object
Wage                 int32
Release Clause      object
Attacking            int64
Crossing             int64
Finishing            int64
Heading Accuracy     int64
Short Passing        int64
Volleys              int64
Skill                int64
Dribbling            int64
Curve                int64
FK Accuracy          int64
Long Passing         int64
Ball Control         int64
Movement             int64
Acceleration         int64
Sprint Speed         int64
Agility              int64
Reactions            int64
B

In [12]:
df.to_csv("fifa_21_cleand_data.csv", header=True)
FileLink("fifa_21_cleand_data.csv")

C:\Users\bkpug\PycharmProjects\AutoIT2\fifa_21_cleand_data.csv